
**Homework #2. linear neural network**

모델을 구현하여 성능을 측정하는 과제입니다.
(해당 과제는 scratch로 구현하는 과제입니다.)

학습에 사용될 dataset은 아래의 첨부파일을 다운로드 받으시면 됩니다.


**얼굴 사진을 통한 나이 예측 모델 학습**

  - 1-110살의 얼굴 사진으로 이루어져 있습니다. 나이의 경우 파일명의 제일 앞에 위치하여 있습니다.

  - 각 사진과 파일명에 적힌 나이를 이용하여 구현하시면 됩니다.

###이번 실습의 학습 목표 입니다.
1. split Dataset (trainset, validationset)
2. loss function 구현 (MSE, Mean Squared Error)
3. linear neural network(linear regression network)
   (필요에 따라 csv파일을 만들어서 사용하는 것 가능)
4. forward, optimizer등 필요한 기능 직접 구현
5. hyperparameter 직접 설정
6. feature importance map

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%matplotlib inline
import torch


* Dataset

In [24]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
import torch.optim as optim

In [98]:

class MyDataset(Dataset):
    def __init__(self, dataset_path, transform=transform):
        self.dataset_dir = dataset_path
        self.transform = transform
        self.image_files = [f for f in os.listdir(dataset_path) if f[-4:] == '.jpg']

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        age_data = int(image_file.split('_')[0])
        label = torch.tensor(age_data, dtype=torch.uint8)

        image = cv2.imread(os.path.join(self.dataset_dir, image_file))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)
            image = image.view(-1)

        return image, label

In [99]:
dataset_dir = '/content/drive/MyDrive/23_2_TA_deep_learning/tr/'
transform = transforms.Compose([transforms.ToTensor()])
dataset = MyDataset(dataset_dir, transform=transform)

def split_dataset(dataset, train_ratio = 0.8):
    total_length = len(dataset)
    train_length = int(total_length * train_ratio)
    val_length = total_length - train_length

    train_indices = list(range(train_length))
    val_indices = list(range(train_length, total_length))

    train_set = Subset(dataset, train_indices)
    val_set = Subset(dataset, val_indices)

    return train_set, val_set

train_set, val_set = split_dataset(dataset, train_ratio = 0.8)

train_loader = DataLoader(train_set, batch_size = 32, shuffle = True)
valid_loader = DataLoader(val_set, batch_size = 32, shuffle = False)

* Model

In [100]:
class LinearRegression(nn.Module):
    """The linear regression model implemented from scratch."""
    def __init__(self):
      super(LinearRegression, self).__init__()
      self.linear = nn.Linear(120000, 1)

    def forward(self,x):
      return self.linear(x)


In [149]:
class Engine:
  def __init__(self, net, epoch, optim, lr):
    self.net = net
    self.epoch = epoch
    self.optim = optim
    self.lr = lr

  def MSE(self, x_hat, x):
      y = ((x - x_hat)**2).mean()
      return y

  def run(self,train,val):
    for i in range(self.epoch):
      for (image,label), (val_image, val_label) in zip(train,val):
        #train
        self.net.train()
        self.optim.zero_grad()
        output = self.net(image)
        loss = self.MSE(output, label)
        loss.backward()
        self.optim.step()

        #val
        self.net.eval()
        output = self.net(val_image)
        val_loss = self.MSE(output, val_label)

        print(f"Epoch {i}/{epoch} : Train Loss : {loss.item()}, Val Loss : {val_loss.item()}")

* Hyperparameter

In [147]:
lr = 1e-6
epoch = 50

In [150]:
net = LinearRegression()
sgd = optim.SGD(net.parameters(), lr=lr)
model = Engine(net, epoch, sgd, lr)
model.run(train_loader, valid_loader)

Epoch 0/50 : Train Loss : 198.49215698242188, Val Loss : 0.12672273814678192
Epoch 0/50 : Train Loss : 178.1515655517578, Val Loss : 1.1998366117477417
Epoch 0/50 : Train Loss : 219.7683868408203, Val Loss : 4.284523010253906
Epoch 0/50 : Train Loss : 172.91029357910156, Val Loss : 17.86040496826172
Epoch 0/50 : Train Loss : 154.9232177734375, Val Loss : 36.81590270996094
Epoch 0/50 : Train Loss : 157.87205505371094, Val Loss : 310.9743347167969
Epoch 0/50 : Train Loss : 153.535400390625, Val Loss : 371.4501037597656
Epoch 0/50 : Train Loss : 102.83060455322266, Val Loss : 22.909423828125
Epoch 0/50 : Train Loss : 120.1861572265625, Val Loss : 35.539981842041016
Epoch 0/50 : Train Loss : 147.91506958007812, Val Loss : 24.910125732421875
Epoch 0/50 : Train Loss : 132.59567260742188, Val Loss : 37.672813415527344
Epoch 0/50 : Train Loss : 103.17378234863281, Val Loss : 33.1226806640625
Epoch 0/50 : Train Loss : 101.6976318359375, Val Loss : 33.16748809814453
Epoch 0/50 : Train Loss : 93.

KeyboardInterrupt: ignored